In [80]:
from __future__ import annotations
import functools
import itertools
import re

from typing import Union


## Utils

In [81]:
# custom types
Char = str
Dataset = list[str]

In [82]:
input_dir = 'input/'
    
def input_for(day: int) -> Dataset:
    try:
        with(open(f'input/day-{day}.txt', 'r')) as file:
            return [line.strip() for line in file ]
    except FileNotFoundError:
        print(f"Input file for day {day} not found")
        

def peek(dataset: Dataset, size: int = 5) -> Union[str, Dataset]:
    if len(dataset) > 1:
        return dataset[:size]
    if len(dataset) == 1:
        return dataset[0][:size]
    else:
        return []

def count(elements: list, predicate=bool) -> int:
    return sum(1 for each in elements if predicate(each))

def any_of(elements: list, predicate=bool) -> bool:
    return next((True for elements in elements if predicate(elements)), False)

def all_of(elements: list, predicate=bool) -> bool:
    return not any_of(elements, lambda x: not predicate(x))

log_level = 0
def log(message: str, level: int):
    if level <= log_level:
        print(message)

## Day 1

In [83]:
# input parsing

day1 = input_for(1)[0]
peek(day1)

'()()('

In [84]:
# part 1

def find_floor(input_values: Dataset) -> int:
    floor = 0
    for each in input_values:
        if each == '(':
            floor += 1
        elif each == ')':
            floor -= 1
        else:
            print('unknown character ' + each)
    return floor

find_floor(day1)

280

In [85]:
# part 2

def find_basement(input_values: Dataset) -> int:
    floor = 0
    for index, each in enumerate(input_values):
        if each == '(': floor += 1
        elif each == ')': floor -= 1
        else: print('unknown character ' + each)
        if floor == -1:
            return index + 1

find_basement(day1)

1797

## Day 2

In [86]:
# input parsing

def parse_line(line: str) -> (int, int, int):
    return [int(each) for each in line.split('x')]

day2 = [parse_line(line) for line in input_for(2)]
peek(day2)

[[20, 3, 11], [15, 27, 5], [6, 29, 7], [30, 15, 9], [19, 29, 21]]

In [87]:
# part 1

def needed_wrap_for(dimensions: (int, int, int)) -> int:
    areas = [first * second for (first, second) in itertools.combinations(dimensions, 2)]
    return min(areas) + 2 * sum(areas)

def needed_wrap_for_all(input_dataset: list[(int, int, int)]):
    return sum(map(needed_wrap_for, input_dataset))

needed_wrap_for_all(day2)

1606483

In [88]:
# part 2

def ribbon_length(sizes: (int, int, int)) -> int:
    return sum(sorted(sizes)[:2]) * 2

def bow_length(sizes: (int, int, int)) -> int:
    return sizes[0] * sizes[1] * sizes[2]

def ribbon_for_package(sizes: (int, int, int)) -> int:
    return ribbon_length(sizes) + bow_length(sizes)

def ribbon_for_all_packages(sizes: list[(int, int, int)]) -> int:
    return sum(map(ribbon_for_package, sizes))

ribbon_for_all_packages(day2)

3842356

## Day 3

In [89]:
# input parsing

day3 = input_for(3)[0]
peek(day3)

'>^^v^'

In [90]:
# part 1

directions = {
    '<': (-1, 0),
    '>': (1, 0),
    '^': (0, 1),
    'v': (0, -1)
}

# def visit_house(instruction):
#
#         if instruction == '<':
#             x -= 1
#         elif instruction == '>':
#             x += 1
#         elif instruction == '^':
#             y += 1
#         elif instruction == 'v':
#             y -= 1
#         else:
#             print(f'unmatched character {instruction}')

def visited_houses(instructions):
    
    position = (0, 0)
    
    visited = set()  # starting house
    visited.add(position)

    for instruction in instructions:
        dx, dy = directions[instruction]
        position = (position[0] + dx, position[1] + dy)
        
        visited.add(position)
    
    return visited

len(visited_houses(day3))

2592

In [91]:
# part 2

def parallel_visit(dataset: Dataset) -> set:
    return visited_houses(dataset[0::2]).union(visited_houses(dataset[1::2]))

len(parallel_visit(day3))

2360

## Day 4

In [92]:
day4 = 'ckczppom'

In [93]:
# part 1

from hashlib import md5

def hash_miner(key: str, zeros=5) -> int:

    bkey = key.encode()
    
    def hash_for(number: int) -> str:
        return md5(bkey + str(number).encode()).hexdigest()

    match = ''.join(['0' for _ in range(zeros)])
    
    for current in itertools.count(0):
        if hash_for(current)[:zeros] == match:
            return current

            
hash_miner(day4)

117946

In [94]:
# part 2

hash_miner(day4, zeros=6)

3938038

## Day 5

In [95]:
# input parsing

day5 = input_for(5)
peek(day5)

['zgsnvdmlfuplrubt',
 'vlhagaovgqjmgvwq',
 'ffumlmqwfcsyqpss',
 'zztdcqzqddaazdjp',
 'eavfzjajkjesnlsb']

In [96]:
# part 1

def day5_1(dataset: Dataset):
    rules_1 = (  # all the rules must be satisfied
        lambda s: len(list(filter(lambda c: c in 'aeiou', s))) >= 3,
        lambda s: re.compile(r'(.)\1').search(s) is not None,
        lambda s: next(filter(lambda x: x, (a == b for a, b in list(zip(s,s[1:])))), False),

        lambda s: all_of(['ab', 'cd', 'pq', 'xy'], lambda x: x not in s)
    )

    def validate_string_1(input_value: str) -> bool:
        return all_of(rules_1, lambda p: p(input_value))

    return count(dataset, validate_string_1)

day5_1(day5)

238

In [97]:
# part 2

def day5_2(dataset: Dataset):
    rules_2 = (
        lambda s: re.compile(r'(.)(.).*\1\2').search(s) is not None,
        lambda s: re.compile(r'(.).\1').search(s) is not None
    )

    def validate_string_2(input_value: str) -> bool:
        return all_of(rules_2, lambda p: p(input_value))

    def validate_all(dataset: Dataset) -> int:
        return count(dataset, validate_string_2)

    return validate_all(dataset)

day5_2(day5)

69

## Day 6

In [98]:
# input parsing

day6 = input_for(6)
peek(day6)

['turn on 489,959 through 759,964',
 'turn off 820,516 through 871,914',
 'turn off 427,423 through 929,502',
 'turn on 774,14 through 977,877',
 'turn on 410,146 through 864,337']

In [99]:
# part 1

# log_level = 0
Point = tuple[int, int]

def day6_1(instructions: Dataset) -> int:
    
    lights = {}
    
    def switch(action: str, start: Point, end: Point):
        log(f'action {action} from {start} through {end}', 1)
        for x in range(int(start[0]), int(end[0]) + 1):
            for y in range(int(start[1]), int(end[1]) + 1):
                if action == 'on':
                    turn_on(x, y)
                elif action == 'off':
                    turn_off(x, y)
                elif action == 'toggle':
                    toggle(x, y)

    def turn_on(x: int, y: int):
        lights[x, y] = True

    def turn_off(x: int, y: int):
        if (x, y) in lights:
            del lights[x, y]
    
    def toggle(x: int, y: int):
        if (x, y) in lights:
            turn_off(x, y)
        else:
            turn_on(x, y)
    
    
    def parse_instruction(instruction: str):
        tokens = instruction.split()
        if tokens[0] == 'toggle': switch('toggle', tokens[1].split(','), tokens[3].split(','))
        elif tokens[1] == 'on': switch('on', tokens[2].split(','), tokens[4].split(','))
        elif tokens[1] == 'off': switch('off', tokens[2].split(','), tokens[4].split(','))
        else: print(f'error parsing command: {instruction}')
#         print(tokens)
    
    for instruction in instructions:
        parse_instruction(instruction)
        log(f'{len(lights)} lights are on', 1)
        
    return len(lights)

day6_1(day6)
# list(map(lambda line: line.split(' ')[:2], day6))

569999

In [100]:
# part 2

def day6_2(instructions: Dataset) -> int:
    
    lights = {}
    
    def switch(action: str, start: Point, end: Point):
        log(f'action {action} from {start} through {end}', 1)
        for x in range(int(start[0]), int(end[0]) + 1):
            for y in range(int(start[1]), int(end[1]) + 1):
                if action == 'on':
                    turn_on(x, y)
                elif action == 'off':
                    turn_off(x, y)
                elif action == 'toggle':
                    toggle(x, y)

    def turn_on(x: int, y: int):
        lights[x, y] = lights.get((x, y), 0) + 1

    def turn_off(x: int, y: int):
        lights[x, y] = max(0, lights.get((x, y), 0) - 1)

    def toggle(x: int, y: int):
        lights[x, y] = lights.get((x, y), 0) + 2
    
    
    def parse_instruction(instruction: str):
        tokens = instruction.split()
        if tokens[0] == 'toggle': switch('toggle', tokens[1].split(','), tokens[3].split(','))
        elif tokens[1] == 'on': switch('on', tokens[2].split(','), tokens[4].split(','))
        elif tokens[1] == 'off': switch('off', tokens[2].split(','), tokens[4].split(','))
        else: print(f'error parsing command: {instruction}')
    
    for instruction in instructions:
        parse_instruction(instruction)
        log(f'{len(lights)} lights are on', 1)
        
    return sum(lights.values())

day6_2(day6)

17836115

# Day 7

In [101]:
# input parsing

day7 = input_for(7)
peek(day7)

['lf AND lq -> ls',
 'iu RSHIFT 1 -> jn',
 'bo OR bu -> bv',
 'gj RSHIFT 1 -> hc',
 'et RSHIFT 2 -> eu']

In [102]:
# part 1

def day7_1(dataset: Dataset, target='a', rewrite=False):

    graph = {}
    memo = {}

    def var(name: str) -> callable:
        if name not in memo:
            memo[name] = graph[name]()

        return memo[name]

    def expr(expr: str) -> callable:
        if expr.isdigit():
            return lambda: int(expr)
        else:
            return lambda: var(expr)

    def build_graph(instructions: Dataset):
        for instruction in instructions:
            parse_line(instruction)

    def parse_line(instruction: str):
        left, right = instruction.split(' -> ')
        if not rewrite:
            assert right not in graph  # only one definition for variable
        graph[right] = parse_instruction(left)

    def parse_instruction(instruction: str) -> callable:
        tokens = instruction.split()
        if len(tokens) == 1:  # 123 -> x
            return expr(tokens[0])

        if len(tokens) == 2 and tokens[0] == 'NOT':  # NOT e -> f
            return lambda: 65_535 ^ var(tokens[1])  # negate all the 16 bits

        elif len(tokens) == 3:
            left = expr(tokens[0])
            operator = tokens[1]
            right = expr(tokens[2])
            if operator == 'AND':  # x AND y -> z
                return lambda: left() & right()
            elif operator == 'OR':  # x OR y -> z
                return lambda: left() | right()
            elif operator == 'LSHIFT':  # x OR y -> z
                return lambda: left() << right()
            elif operator == 'RSHIFT':  # x OR y -> z
                return lambda: left() >> right()

        else:
            print(f'error parsing instruction: {instruction}')

    build_graph(dataset)
    result = graph[target]()
    print(memo)

    return result


day7_1(day7)

{'c': 0, 't': 0, 'b': 19138, 'd': 4784, 'e': 2392, 'f': 598, 'g': 2910, 'h': 80, 'i': 65455, 'j': 2830, 'k': 7102, 'l': 512, 'm': 65023, 'n': 6590, 'o': 23550, 'p': 2178, 'q': 63357, 'r': 21372, 's': 0, 'u': 0, 'ao': 0, 'v': 9569, 'w': 0, 'x': 9569, 'y': 2392, 'z': 1196, 'aa': 299, 'ab': 1455, 'ac': 40, 'ad': 65495, 'ae': 1415, 'af': 3551, 'ag': 256, 'ah': 65279, 'ai': 3295, 'aj': 11775, 'ak': 1089, 'al': 64446, 'am': 10686, 'an': 0, 'ap': 0, 'bj': 0, 'aq': 4784, 'ar': 0, 'as': 4784, 'at': 1196, 'au': 598, 'av': 149, 'aw': 727, 'ax': 20, 'ay': 65515, 'az': 707, 'ba': 1775, 'bb': 128, 'bc': 65407, 'bd': 1647, 'be': 5887, 'bf': 544, 'bg': 64991, 'bh': 5343, 'bi': 1, 'bk': 1, 'ce': 2, 'bl': 2392, 'bm': 32768, 'bn': 35160, 'bo': 8790, 'bp': 4395, 'bq': 1098, 'br': 5483, 'bs': 10, 'bt': 65525, 'bu': 5473, 'bv': 14199, 'bw': 64, 'bx': 65471, 'by': 14135, 'bz': 49023, 'ca': 272, 'cb': 65263, 'cc': 48751, 'cd': 1, 'cf': 3, 'cz': 6, 'cg': 17580, 'ch': 32768, 'ci': 50348, 'cj': 12587, 'ck': 6293

16076

In [103]:
# part 2

def day7_2(dataset: Dataset, target: str = 'a') -> int:
    return day7_1(dataset, target=target, rewrite=True)

day7_2(day7 + ['16076 -> b'])

{'c': 0, 't': 0, 'b': 16076, 'd': 4019, 'e': 2009, 'f': 502, 'g': 2047, 'h': 464, 'i': 65071, 'j': 1583, 'k': 4031, 'l': 1571, 'm': 63964, 'n': 2460, 'o': 16348, 'p': 2188, 'q': 63347, 'r': 14160, 's': 0, 'u': 0, 'ao': 0, 'v': 8038, 'w': 0, 'x': 8038, 'y': 2009, 'z': 1004, 'aa': 251, 'ab': 1023, 'ac': 232, 'ad': 65303, 'ae': 791, 'af': 2015, 'ag': 785, 'ah': 64750, 'ai': 1230, 'aj': 8174, 'ak': 1094, 'al': 64441, 'am': 7080, 'an': 0, 'ap': 0, 'bj': 0, 'aq': 4019, 'ar': 0, 'as': 4019, 'at': 1004, 'au': 502, 'av': 125, 'aw': 511, 'ax': 116, 'ay': 65419, 'az': 395, 'ba': 1007, 'bb': 392, 'bc': 65143, 'bd': 615, 'be': 4087, 'bf': 547, 'bg': 64988, 'bh': 3540, 'bi': 0, 'bk': 0, 'ce': 0, 'bl': 2009, 'bm': 0, 'bn': 2009, 'bo': 502, 'bp': 251, 'bq': 62, 'br': 255, 'bs': 58, 'bt': 65477, 'bu': 197, 'bv': 503, 'bw': 196, 'bx': 65339, 'by': 307, 'bz': 2043, 'ca': 273, 'cb': 65262, 'cc': 1770, 'cd': 0, 'cf': 0, 'cz': 0, 'cg': 1004, 'ch': 0, 'ci': 1004, 'cj': 251, 'ck': 125, 'cl': 31, 'cm': 127, 'c

2797

In [104]:
# input parsing

day8 = input_for(8)
peek(day8)

['"qxfcsmh"',
 '"ffsfyxbyuhqkpwatkjgudo"',
 '"byc\\x9dyxuafof\\\\\\xa6uf\\\\axfozomj\\\\olh\\x6a"',
 '"jtqvz"',
 '"uzezxa\\"jgbmojtwyfbfguz"']

In [107]:
# part 1
def memory(line: str, replace_rules):
    return functools.reduce(lambda s, rule: re.sub(rule[0], rule[1], s),
                                replace_rules.items(), line)


def day8_1(dataset: Dataset):

    replace_rules = {  # dictionaries are now sorted in python
        r'\\x[0-9a-f]{2}': 'U',
        r'\\[\\"]': 'E',
        r'"': ''
    }

    def diff(line: str) -> int:
        encoded = memory(line, replace_rules)
        # print(f'{line} -> {encoded} — {len(line)}, {len(encoded)}')
        return len(line) - len(encoded)

    return sum(map(diff, dataset))


day8_1(day8)

1350

In [108]:
def day8_2(dataset: Dataset) -> int:

    replace_rules = {
        r'\\x[0-9a-f]{2}': r'UUxUU',
        r'\\[\\"]': 'EEEE',
        r'"': r'\"'
    }

    def diff(line: str) -> int:
        encoded = "\"" + memory(line, replace_rules) + "\""
        # print(f'{line} -> {encoded} — {len(line)}, {len(encoded)}')
        return len(encoded) - len(line)

    return sum(map(diff, dataset))


day8_2(day8)

2085